In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Load dataset
df = pd.read_csv('../raw_data/creditcard.csv')

# Preview the first 5 rows
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### Preprocessing the Data for Deep Learning
### Clean, Scale, Split, Reshape

In [3]:
# 1. Check for missing values
print("Missing values:\n", df.isnull().sum())

# 2. Feature scaling: scale 'Amount' (important for model convergence)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df['Amount'] = scaler.fit_transform(df[['Amount']])

# 3. Drop the 'Time' column (not needed)
df = df.drop(['Time'], axis=1)

# 4. Split features and labels
X = df.drop('Class', axis=1)
y = df['Class']

# 5. Train-test split with stratification (to preserve class imbalance)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

Missing values:
 Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64
Training samples: 227845
Testing samples: 56962


### Reshape for CNN+LSTM

In [4]:
# CNN/LSTM expect input shape: (samples, time_steps, features)
X_train_reshaped = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_reshaped = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

print("Train shape:", X_train_reshaped.shape)
print("Test shape:", X_test_reshaped.shape)

Train shape: (227845, 1, 29)
Test shape: (56962, 1, 29)


## What This Above Code Does:
#### Step	Why It Matters
#### Scaling	- Neural nets need scaled inputs to learn faster
#### Drop Time - Not useful for modeling (leaks temporal data)
#### Reshape - Required input for CNN+LSTM layers
#### Stratified Split - Keeps rare fraud cases balanced across train/test

### Build & Train the Model
### Define the CNN + LSTM Model

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout

# Build the model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(1, X_train.shape[1])))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model summary
model.summary()

/Users/jaykotia/Desktop/finsight-ai/venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1, 64)          │         1,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,009 (136.75 KB)

 Trainable params: 35,009 (136.75 KB)

 Non-trainable params: 0 (0.00 B)

### Train the Model

In [6]:
# Train the model
history = model.fit(
    X_train_reshaped, y_train,
    epochs=5,                     # You can increase this later
    batch_size=2048,
    validation_data=(X_test_reshaped, y_test),
    verbose=1
)

Epoch 1/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9790 - loss: 0.2712 - val_accuracy: 0.9983 - val_loss: 0.0100
Epoch 2/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9984 - loss: 0.0088 - val_accuracy: 0.9991 - val_loss: 0.0043
Epoch 3/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9993 - loss: 0.0045 - val_accuracy: 0.9994 - val_loss: 0.0036
Epoch 4/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9994 - loss: 0.0038 - val_accuracy: 0.9994 - val_loss: 0.0032
Epoch 5/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9994 - loss: 0.0032 - val_accuracy: 0.9994 - val_loss: 0.0030


### Evaluate the Model

In [7]:
# Evaluate model performance
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f"\n🔍 Test Accuracy: {accuracy:.4f}")

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 1s 391us/step - accuracy: 0.9994 - loss: 0.0023

🔍 Test Accuracy: 0.9994


### Save the Trained Model (For API Later)

In [8]:
# Save model to disk
model.save('../models/cnn_lstm_anomaly_detector.h5')
print("✅ Model saved to models/ folder.")

✅ Model saved to models/ folder.
